# Running Marlowe Contracts without Blockchain Transactions

This lecture provides an overview of the low and high levels at which one can create and interact with Marlowe contracts on the blockchain using the Marlowe CLI tool, and how that relates to using Marlowe Run or the Marlowe Playground's simulator. It introduces the `marlowe-cli` tool and includes instructions for installing and using it. Subsequent lectures focus on particular levels of interaction with Marlowe contract using `marlowe-cli`.

## Escrow Contract

*   A buyer wishes to purchase an item.
*   They deposit funds.
*   After they deposit funds they may report a problem with the purchase.
*   If they don’t report a problem, then the funds are released to the seller.
*   If they do report a problem, the seller may agree that there is a problem (in which case the buyer receives a refund) or they may dispute that there is a problem.
*   If the seller disputes the problem, then a mediator decides who receives the funds.
*   The contract has logic to handle situations where a party fails to act in a timely manner.

### Flow Chart for Escrow Example

![Execution paths through the escrow example contract.](diagrams/escrow-flow.png)

### Escrow Example in Marlowe Format

```haskell
When
  [
    Case (Deposit (Role "Francis Beaumont (the seller)") (Role "Thomas Middleton (the buyer)") ada 256)
      ( When
        [
          Case (Choice (ChoiceId "Everything is alright" (Role "Thomas Middleton (the buyer)")) [Bound 0 0])
            Close
        , Case (Choice (ChoiceId "Report problem" (Role "Thomas Middleton (the buyer)")) [Bound 1 1])
            ( Pay (Role "Francis Beaumont (the seller)") (Account (Role "Thomas Middleton (the buyer)")) ada 256 )
            ( When
              [
                Case (Choice (ChoiceId "Confirm problem" (Role "Francis Beaumont (the seller)")) [Bound 1 1])
                  Close
              , Case (Choice (ChoiceId "Dispute problem" (Role "Francis Beaumont (the seller)")) [Bound 0 0])
                ( When
                  [
                    Case (Choice (ChoiceId "Dismiss claim" (Role "Christopher Marlowe (the mediator)")) [Bound 0 0])
                      ( Pay (Role "Thomas Middleton (the buyer)") (Account (Role "Francis Beaumont (the seller)")) ada 256 )
                      Close
                  , Case (Choice (ChoiceId "Confirm claim" (Role "Christopher Marlowe (the mediator)")) [Bound 1 1])
                      Close
                  ]
                  (SlotParam "Mediation deadline")
                  Close
                )
              ]
              (SlotParam "Dispute deadline")
            ) Close
        ]
        (SlotParam "Complaint deadline")
        Close
      )
  ]
  (SlotParam "Payment deadline")
  Close
```

### Escrow Example in Blockly

![Blockly representation of escrow example contract, split into three columns.](diagrams/escrow-blockly.png)

### Four of the Eight Pathways through the Escrow Contract

![Four possible executions of the escrow contract.](diagrams/escrow-paths.png)

## Scenario

![Overview of escrow example.](diagrams/escrow-detailed-setup.png)

## Select the Parameters for the Contract

First use some environment variables to store some values

In [7]:
INITIAL_LOVELACE=3000000             # The creation transaction will deposit 3₳.
PRICE=50000000                       # The prices of the item is 50₳.

SELLER_ROLE=FB                       # Francis Beaumont (FB) is the seller.
BUYER_ROLE=TM                        # Thomas Middleton is the buyer.
MEDIATOR_ROLE=CM                     # Christopher Marlowe is the mediator.

NOW=$(($(date -u +%s)*1000))         # The current time in POSIX milliseconds.
HOUR=$((60*60*1000))                 # One hour, in POSIX milliseconds.
PAYMENT_DEADLINE=$((NOW+10*HOUR))    # The payment deadline, ten hours from now.
COMPLAINT_DEADLINE=$((NOW+12*HOUR))  # The complaint deadline, twelve hours from now.
DISPUTE_DEADLINE=$((NOW+14*HOUR))    # The dispute deadline, fourteen hours from now.
MEDIATION_DEADLINE=$((NOW+16*HOUR))  # The mediation deadline, sixteen hours from now.

## Create the Contract and Its Initial State

Now create the contract:
*   The contract is stored in the JSON file *tx-1.contract*.
*   The initial state is stored in the JSON file *tx-1.state*.

In [9]:
marlowe-cli template escrow --minimum-ada "$INITIAL_LOVELACE"          \
                            --price "$PRICE"                           \
                            --seller "Role=$SELLER_ROLE"               \
                            --buyer "Role=$BUYER_ROLE"                 \
                            --mediator "Role=$MEDIATOR_ROLE"           \
                            --payment-deadline "$PAYMENT_DEADLINE"     \
                            --complaint-deadline "$COMPLAINT_DEADLINE" \
                            --dispute-deadline "$DISPUTE_DEADLINE"     \
                            --mediation-deadline "$MEDIATION_DEADLINE" \
                            --out-contract-file tx-1.contract          \
                            --out-state-file    tx-1.state


## Other Lectures

*   Overview of Marlowe CLI
*   Installing Marlowe CLI and Associated Tools
*   ~Running Marlowe Contracts without Blockchain Transactions~
*   Running Marlowe Contacts on the Blockchain
*   Running Marlowe Contracts with the Plutus Application Backend (PAB)
*   Reference for Marlowe CLI Commands